# Who is friends with Epstein?

<img src="images/ep.jpg" alt="Jeff and island" height="200">


**Disclaimer** 

Much like Thomas Massie said: 

- I am not suicidal. 
- I eat healthy food. 
- The brakes on my car are in good shape. 
- I practice good trigger discipline and never point a gun at anyone, including myself. 
- There are no deep pools of water in my life and I’m a great swimmer.

In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from shapely.geometry import Point, Polygon

# This is all fake data and just for fun


In [2]:
start_date = datetime(2026, 3, 1)
days = 14

def jitter(lat, lon):
    return (
        lat + np.random.uniform(-0.0004, 0.0004),
        lon + np.random.uniform(-0.0004, 0.0004)
    )

#  Core Locations 
locations = {
    "island": (18.3005, -64.8250, "Charlotte Amalie", "vi", "00802", "Atlantic Standard Time", "-04:00"),
    "ranch": (35.2665, -105.9695, "Santa Fe", "nm", "87501", "Mountain Time", "-07:00"),
    "nyc": (40.77156, -73.96645, "New York", "ny", "10021", "Eastern Time", "-05:00"),
    "miami": (25.7617, -80.1918, "Miami", "fl", "33101", "Eastern Time", "-05:00"),
    "denver": (39.7392, -104.9903, "Denver", "co", "80202", "Mountain Time", "-07:00"),
    "dallas": (32.7767, -96.7970, "Dallas", "tx", "75201", "Central Time", "-06:00"),
    "atlanta": (33.7490, -84.3880, "Atlanta", "ga", "30301", "Eastern Time", "-05:00"),
}

primary_people = ["person_A", "person_B"]
visitors = [f"visitor_{i}" for i in range(1, 11)]

all_people = primary_people + visitors

#  Assign Stable Home Locations 
home_map = {
    "person_A": "miami",
    "person_B": "denver",
}

for v in visitors:
    home_map[v] = random.choice(["miami", "denver", "dallas", "atlanta"])

#  Event Generators 

all_events = []

def create_event(device_id, base_date, loc_key, hours=8):
    lat, lon, city, region, postal, tz, offset = locations[loc_key]
    start = base_date + timedelta(hours=random.randint(7, 10))

    for i in range(hours * 2):  # 30-min intervals
        jlat, jlon = jitter(lat, lon)
        all_events.append({
            "device_id": device_id,
            "event_time": start + timedelta(minutes=30*i),
            "latitude": jlat,
            "longitude": jlon,
            "city": city,
            "region": region,
            "country": "usa",
            "two_letter_country": "us",
            "postal_code": postal,
            "timezone_name": tz,
            "gmt_offset": offset
        })

def create_night_home(device_id, base_date, loc_key):
    lat, lon, city, region, postal, tz, offset = locations[loc_key]
    start = base_date + timedelta(hours=22)  # 10 PM

    for i in range(6):  # 10 PM – 4 AM
        jlat, jlon = jitter(lat, lon)
        all_events.append({
            "device_id": device_id,
            "event_time": start + timedelta(hours=i),
            "latitude": jlat,
            "longitude": jlon,
            "city": city,
            "region": region,
            "country": "usa",
            "two_letter_country": "us",
            "postal_code": postal,
            "timezone_name": tz,
            "gmt_offset": offset
        })

#  Generate Data 
for d in range(days):
    date = start_date + timedelta(days=d)

    #  Primary Individuals 
    create_event("person_A", date, "island", hours=random.randint(7,10))
    create_event("person_A", date, home_map["person_A"], hours=4)

    create_event("person_B", date, "ranch", hours=random.randint(7,10))
    create_event("person_B", date, home_map["person_B"], hours=4)

    #  Visitors 
    for v in visitors:
        work = random.choice(["nyc", "denver", "dallas"])

        create_event(v, date, home_map[v], hours=3)
        create_event(v, date, work, hours=4)

        if random.random() < 0.15:
            poi = random.choice(["island", "ranch", "nyc"])
            create_event(v, date, poi, hours=random.randint(1,3))

    #  Night Home for ALL devices 
    for person in all_people:
        create_night_home(person, date, home_map[person])

#  Finalize 
df = pd.DataFrame(all_events)
df = df.sort_values("event_time").reset_index(drop=True)

df.to_csv("organic_mobility_data.csv", index=False)

print(f"Rows created: {len(df)}")


Rows created: 3754


# 😉 *Absolutely do not add your own real data here* 😉

If you break the rules here you might have to adjust the column names to fit code below. 


In [3]:
df = pd.read_csv("organic_mobility_data.csv")

## Little Saint James Island Geofence

<img src="images/lsj.avif" alt="Little Saint James Island" height="200">

In [4]:
island_poly = Polygon([
    (-64.830515, 18.304551),
    (-64.820305, 18.304482),
    (-64.819889, 18.296081),
    (-64.829742, 18.296586),
])



## Zorro Ranch Geofence

<img src="images/zorro.jpg" alt="Zorro Ranch" height="200">

In [5]:
ranch_poly = Polygon([
    (-105.970426, 35.268219),
    (-105.967583, 35.267702),
    (-105.968399, 35.265197),
    (-105.971199, 35.265871),
])


## Herbert N. Straus House Geofence

<img src="images/house.jpg" alt="Zorro Ranch"  height="200">

In [6]:
nyc_poly = Polygon([
    (-73.96665, 40.77176),
    (-73.96625, 40.77176),
    (-73.96625, 40.77136),
    (-73.96665, 40.77136),
])

### Checking the data to see who has visited
Will return new columns with boolean values to if they visited or not. 

| is_island | is_ranch | is_nyc |
| :-:    | :-----: | :-----:  |
| True      | False    | True   |
| False     | False    | True   |

In [7]:
def check_geofences(row):
    point = Point(row["longitude"], row["latitude"])
    return pd.Series({
        "is_island": island_poly.contains(point),
        "is_ranch": ranch_poly.contains(point),
        "is_nyc": nyc_poly.contains(point)
    })

geo_flags = df.apply(check_geofences, axis=1)
df = pd.concat([df, geo_flags], axis=1)
df.head()

,device_id,event_time,latitude,longitude,city,region,country,two_letter_country,postal_code,timezone_name,gmt_offset,is_island,is_ranch,is_nyc
0,visitor_5,2026-03-01 07:00:00,32.776339,-96.797000,Dallas,tx,usa,us,75201,Central Time,-06:00,False,False,False
1,visitor_8,2026-03-01 07:00:00,39.739342,-104.990052,Denver,co,usa,us,80202,Mountain Time,-07:00,False,False,False
2,visitor_3,2026-03-01 07:00:00,25.761339,-80.191517,Miami,fl,usa,us,33101,Eastern Time,-05:00,False,False,False
3,visitor_2,2026-03-01 07:00:00,39.739184,-104.990341,Denver,co,usa,us,80202,Mountain Time,-07:00,False,False,False
4,visitor_5,2026-03-01 07:00:00,39.739224,-104.990467,Denver,co,usa,us,80202,Mountain Time,-07:00,False,False,False


### Grouping People Who Visited All 3
Alarm bells! 

In [8]:
visited_all_three = (
    df.groupby("device_id")[["is_island", "is_ranch", "is_nyc"]]
    .any()
    .reset_index()
)

visited_all_three = visited_all_three[
    visited_all_three["is_island"] &
    visited_all_three["is_ranch"] &
    visited_all_three["is_nyc"]
]

visited_all_three 

,device_id,is_island,is_ranch,is_nyc
2,visitor_1,True,True,True


### This allows you to filter time frames
We are then able to likely find Work or home locations. 



In [9]:
import pandas as pd

def get_main_location(df, start_hour, end_hour, min_hours=0):
    df = df.copy()
    df["event_time"] = pd.to_datetime(df["event_time"])
    df["hour"] = df["event_time"].dt.hour
    
    # 1. Assign Logical Period Date
    df["period_date"] = df["event_time"].dt.normalize() 
    if end_hour < start_hour:
        # Shift early morning pings (00:00 - end_hour) to the previous day
        mask_early = df["hour"] <= end_hour
        df.loc[mask_early, "period_date"] = (df.loc[mask_early, "event_time"] - pd.Timedelta(days=1)).dt.normalize()
        mask = (df["hour"] >= start_hour) | (df["hour"] <= end_hour)
    else:
        mask = (df["hour"] >= start_hour) & (df["hour"] <= end_hour)

    # 2. Filter data
    period_df = df[mask].copy()
    print(f"DEBUG: Found {len(period_df)} rows for hour range {start_hour}-{end_hour}")
    
    if period_df.empty:
        return pd.DataFrame()

    # 3. Calculate time spent (Duration)
    period_df = period_df.sort_values(["device_id", "period_date", "event_time"])
    period_df["next_time"] = period_df.groupby(["device_id", "period_date"])["event_time"].shift(-1)
    period_df["duration"] = (period_df["next_time"] - period_df["event_time"]).dt.total_seconds() / 3600
    
    # Fill single-ping durations with 1 hour so they aren't ignored
    period_df["duration"] = period_df["duration"].fillna(1.0)
    period_df.loc[period_df["duration"] <= 0, "duration"] = 1.0

    # 4. Find the most frequent location per person, per night
    agg = (
        period_df.groupby(["device_id", "period_date", "city", "latitude", "longitude"], dropna=False)
        .agg({"duration": "sum"})
        .reset_index()
    )
    
    result = (
        agg.sort_values("duration", ascending=False)
        .groupby(["device_id", "period_date"], as_index=False)
        .first()
    )
    
    result["period_date"] = result["period_date"].dt.date
    return result

#  test: Re-load data to ensure it hasn't been filtered 
# df_fresh = pd.read_csv('organic_mobility_data.csv')

# # Test Sleep (Overnight)
# print(" Testing Sleep (21:00-06:00) ")
# sleep_locations = get_main_location(df_fresh, 21, 6)
# print(sleep_locations.head())

# # Test Early Morning (Standard range)
# print("\n Testing Early Morning (01:00-06:00) ")
# early_morning = get_main_location(df_fresh, 1, 6)
# print(early_morning.head())

### Adjust start and end hours as needed 

In [10]:
# Sleep locations (9PM–6AM)
sleep_locations = get_main_location(df, start_hour=21, end_hour=6, min_hours=0)

DEBUG: Found 1008 rows for hour range 21-6


In [11]:
sleep_locations

,device_id,period_date,city,latitude,longitude,duration
0,person_A,2026-03-01,Miami,25.761393,-80.192053,1.0
1,person_A,2026-03-02,Miami,25.761449,-80.191522,1.0
2,person_A,2026-03-03,Miami,25.761591,-80.191916,1.0
3,person_A,2026-03-04,Miami,25.761317,-80.191533,1.0
4,person_A,2026-03-05,Miami,25.762046,-80.191980,1.0
...,...,...,...,...,...,...
163,visitor_9,2026-03-10,Miami,25.761599,-80.191685,1.0
164,visitor_9,2026-03-11,Miami,25.761732,-80.191723,1.0
165,visitor_9,2026-03-12,Miami,25.761475,-80.191855,1.0
166,visitor_9,2026-03-13,Miami,25.761305,-80.191550,1.0


In [12]:
visitor_1 = sleep_locations[sleep_locations["device_id"] == "visitor_1"]
visitor_1

,device_id,period_date,city,latitude,longitude,duration
28,visitor_1,2026-03-01,Miami,25.761863,-80.191811,1.0
29,visitor_1,2026-03-02,Miami,25.761305,-80.191928,1.0
30,visitor_1,2026-03-03,Miami,25.761306,-80.191402,1.0
31,visitor_1,2026-03-04,Miami,25.761426,-80.191989,1.0
32,visitor_1,2026-03-05,Miami,25.761389,-80.191769,1.0
33,visitor_1,2026-03-06,Miami,25.761422,-80.191921,1.0
34,visitor_1,2026-03-07,Miami,25.761321,-80.191806,1.0
35,visitor_1,2026-03-08,Miami,25.761373,-80.191557,1.0
36,visitor_1,2026-03-09,Miami,25.761813,-80.192171,1.0
37,visitor_1,2026-03-10,Miami,25.761371,-80.191938,1.0


In [13]:
# Work locations: 09:00–17:00
work_locations = get_main_location(df, start_hour=9, end_hour=17, min_hours=0)

DEBUG: Found 2174 rows for hour range 9-17


In [14]:
work_locations

,device_id,period_date,city,latitude,longitude,duration
0,person_A,2026-03-01,Charlotte Amalie,18.300147,-64.824792,1.0
1,person_A,2026-03-02,Miami,25.762081,-80.191882,1.0
2,person_A,2026-03-03,Charlotte Amalie,18.300395,-64.825327,1.0
3,person_A,2026-03-04,Charlotte Amalie,18.300719,-64.824740,1.0
4,person_A,2026-03-05,Charlotte Amalie,18.300619,-64.824924,1.0
...,...,...,...,...,...,...
163,visitor_9,2026-03-10,Miami,25.761844,-80.192032,1.0
164,visitor_9,2026-03-11,Miami,25.761988,-80.192031,1.0
165,visitor_9,2026-03-12,Miami,25.761423,-80.191496,1.0
166,visitor_9,2026-03-13,Santa Fe,35.266850,-105.969208,1.0


In [15]:
visitor_9 = work_locations[work_locations["device_id"] == "visitor_9"]
visitor_9

,device_id,period_date,city,latitude,longitude,duration
154,visitor_9,2026-03-01,Dallas,32.776557,-96.796924,1.0
155,visitor_9,2026-03-02,Miami,25.761915,-80.191635,1.0
156,visitor_9,2026-03-03,Miami,25.761904,-80.191605,1.0
157,visitor_9,2026-03-04,Miami,25.761910,-80.191651,1.0
158,visitor_9,2026-03-05,New York,40.771493,-73.966690,1.0
159,visitor_9,2026-03-06,Miami,25.761363,-80.191768,1.0
160,visitor_9,2026-03-07,Miami,25.761873,-80.191674,1.0
161,visitor_9,2026-03-08,New York,40.771831,-73.966298,1.0
162,visitor_9,2026-03-09,Denver,39.739585,-104.990447,1.0
163,visitor_9,2026-03-10,Miami,25.761844,-80.192032,1.0


### Use the Above to fill in data below. 

filtering to people who only visited the locations


In [16]:
df = df[
    (df["is_island"]) |
    (df["is_ranch"]) |
    (df["is_nyc"])
].copy()

In [17]:
def found_name(df, device_id, name, name_col="name"):
    df = df.copy()
    # Only create column if it doesn't exist
    if name_col not in df.columns:
        df[name_col] = ""
    
    # Assign name to rows with given device_id
    df.loc[df["device_id"] == device_id, name_col] = name
    
    # Reorder columns to put name before device_id
    cols = df.columns.tolist()
    cols.remove(name_col)
    device_idx = cols.index("device_id")
    new_cols = cols[:device_idx] + [name_col] + cols[device_idx:]
    
    return df[new_cols]


def add_home_column(df, device_id, home_location, col_name="home"):
    df = df.copy()
    if col_name not in df.columns:
        df[col_name] = ""
    df.loc[df["device_id"] == device_id, col_name] = home_location
    
    cols = df.columns.tolist()
    cols.remove(col_name)
    device_idx = cols.index("device_id")
    new_cols = cols[:device_idx] + [col_name] + cols[device_idx:]
    
    return df[new_cols]


def add_work_column(df, device_id, work_location, col_name="work"):
    df = df.copy()
    if col_name not in df.columns:
        df[col_name] = ""
    df.loc[df["device_id"] == device_id, col_name] = work_location
    
    cols = df.columns.tolist()
    cols.remove(col_name)
    device_idx = cols.index("device_id")
    new_cols = cols[:device_idx] + [col_name] + cols[device_idx:]
    
    return df[new_cols]


In [18]:
df = found_name(df, "visitor_9", "Bob Smith")

Run this over and over to add names of people. Keep df, visitor_9 is the value you want to change in the device_id column. Bob Smith is the name you want to add to the name column. 

```python
df = found_name(df, "visitor_9", "Bob Smith")
```

In [19]:
df = found_name(df, "visitor_8", "Tom Johnson")
df = found_name(df, "visitor_7", "Amy Johnson")
df = found_name(df, "visitor_6", "Tom Smithers")
df = found_name(df, "visitor_5", "Tomi Bond")
df = found_name(df, "visitor_4", "Lonnie young")
df = found_name(df, "visitor_3", "Amanda brown")
df = found_name(df, "visitor_2", "Charlie Davis")
df = found_name(df, "visitor_1", "Emily Wilson")
df = found_name(df, "visitor_1", "Bill Hates")

In [20]:
# add more code blocks as needed to fill in your list. 

In [21]:
df = add_home_column(df, "visitor_10", "123 Google St, Mountain View, CA 94043")
df = add_home_column(df, "visitor_9", "1099 Miami Ave, Miami, FL 33130")
df = add_home_column(df, "visitor_8", "456 BP Blvd, Houston, TX 77002")
df = add_home_column(df, "visitor_7", "789 Amazon Rd, Seattle, WA 98109")
df = add_home_column(df, "visitor_6", "101 Facebook Ln, Menlo Park, CA 94025")
df = add_home_column(df, "visitor_5", "202 Apple Way, Cupertino, CA 95014")
df = add_home_column(df, "visitor_4", "303 Tesla Dr, Palo Alto, CA 94304")
df = add_home_column(df, "visitor_3", "404 JPM St, New York, NY 10005")
df = add_home_column(df, "visitor_2", "505 Microsoft Ave, Redmond, WA 98052")
df = add_home_column(df, "visitor_1", "606 Goldman St, New York, NY 10024")

In [22]:
df = add_work_column(df, "visitor_10", "Google")
df = add_work_column(df, "visitor_9", "Goldman Sachs")
df = add_work_column(df, "visitor_8", "BP Oil")
df = add_work_column(df, "visitor_7", "Amazon")
df = add_work_column(df, "visitor_6", "Facebook")
df = add_work_column(df, "visitor_5", "Apple")
df = add_work_column(df, "visitor_4", "Tesla")
df = add_work_column(df, "visitor_3", "JP Morgan")
df = add_work_column(df, "visitor_2", "Microsoft")
df = add_work_column(df, "visitor_1", "Goldman Sachs")


In [23]:
df.head()

,name,home,work,device_id,event_time,latitude,longitude,city,region,country,two_letter_country,postal_code,timezone_name,gmt_offset,is_island,is_ranch,is_nyc
39,,,,person_B,2026-03-01 09:00:00,35.266144,-105.969546,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
54,,,,person_B,2026-03-01 09:30:00,35.266211,-105.969143,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
72,,,,person_A,2026-03-01 10:00:00,18.300166,-64.824617,Charlotte Amalie,vi,usa,us,802,Atlantic Standard Time,-04:00,True,False,False
77,,,,person_B,2026-03-01 10:00:00,35.266544,-105.969676,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
89,,,,person_B,2026-03-01 10:30:00,35.266439,-105.969420,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False


### All the data for named user
Change "Bob Smith" to a different name to get data

In [24]:
bob = df[df["name"] == "Bob Smith"]
bob

,name,home,work,device_id,event_time,latitude,longitude,city,region,country,two_letter_country,postal_code,timezone_name,gmt_offset,is_island,is_ranch,is_nyc
539,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-03 08:00:00,40.771740,-73.966299,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True
669,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-03 11:30:00,40.771444,-73.966507,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True
2021,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-08 11:00:00,40.771511,-73.966556,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True
2985,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-12 08:30:00,40.771732,-73.966636,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True
3032,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-12 10:00:00,40.771463,-73.966395,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True
3270,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-13 09:00:00,35.266402,-105.969606,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
3283,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-13 09:30:00,35.266850,-105.969208,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
3306,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-13 10:00:00,35.266225,-105.969805,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
3326,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-13 10:30:00,35.266498,-105.969429,Santa Fe,nm,usa,us,87501,Mountain Time,-07:00,False,True,False
3523,Bob Smith,"1099 Miami Ave, Miami, FL 33130",Goldman Sachs,visitor_9,2026-03-14 09:00:00,40.771600,-73.966635,New York,ny,usa,us,10021,Eastern Time,-05:00,False,False,True


Once you complete your research run this to generate the list of friends! 

friend_list = df.to_csv(“epstiens_friends.csv”) 



In [25]:
# Replace empty strings (and strings with just spaces) with NaN
df["name"] = df["name"].replace(r'^\s*$', np.nan, regex=True)

# Columns to check for True
cols = ["is_island", "is_ranch", "is_nyc"]

# Filter: any of the flags is True AND name is not null
final_df = df[df[cols].any(axis=1) & df["name"].notna()].copy()

# Group by name and keep only name, home, work
final_df = (
    final_df.groupby("name", as_index=False)[["home", "work"]]
    .first()
)

friends = final_df.to_csv("friends_on_island.csv", index=False)


FIN 


Who was friends of Epstein?